In [1]:
import os 
import csv
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
random.seed(42)

In [3]:
path = '../ce_data/output_512/dataset.csv'
df = pd.read_csv(path)
miss_ratios = pd.DataFrame()
for i in ["1024","2048","4096","8192"]:
    miss_ratios[i] = df['misses_'+i] / df['accesses_'+i]
    df = df.drop(columns=['misses_'+i, 'accesses_'+i, 'l1d_accesses_'+i])
print(miss_ratios)
print(df)

          1024      2048      4096      8192
0     0.890827  0.795337  0.662562  0.123317
1     0.866846  0.765354  0.660919  0.077643
2     0.848553  0.727294  0.641544  0.262694
3     0.617509  0.223128  0.040222  0.001846
4     0.588169  0.200199  0.032464  0.001848
...        ...       ...       ...       ...
6186  0.785621  0.600917  0.436724  0.297468
6187  0.739754  0.549444  0.413295  0.301718
6188  0.934745  0.882877  0.810774  0.632815
6189  0.949653  0.904684  0.823035  0.558143
6190  0.998407  0.998407  0.998407  0.998407

[6191 rows x 4 columns]
                         id         0         1         2        3        4  \
0       400.perlbench-41B_0  50816880  17323312  11868112  7701840  3853504   
1     400.perlbench-41B_436  17361520   6023600   4203744  2591488  1828656   
2       400.perlbench-50B_0  58283520  17249296  14592592  9466576  7204896   
3          401.bzip2-226B_0   3098304   3037808   2698720  2692176  1999568   
4         401.bzip2-226B_54   1623056   

In [4]:
filenames = []
for i in df["id"]:
    nmb = i.split('_')[-1]
    filenames.append(i[:-len(nmb)-1])
df["file_name"] = filenames
filenames = list(set(filenames))
print(len(filenames))   # 189

189


There's a chance that reuse distance histograms within files are similar to each other. This may be due to them being sampled from the same files and therefore generated from the same code, so the histograms will be similar. To avoid this interfering with our process, we'll be moving all the lines that are derived from randomly picked files until they add up to 20% of the dataset. 

In [5]:
collected_rows = 0
files = []
while len(df) * 0.2 > collected_rows:
    rf = filenames[random.randint(0,188)]
    if rf in files: # No duplicates
        continue
    files.append(rf)
    collected_rows += len(df[df["file_name"] == rf])
print(f"The test set consists of {len(files)} files with a total of {collected_rows} rows")

The test set consists of 52 files with a total of 1258 rows


In [6]:
train_rds = df[~df["file_name"].isin(files)]
train_mrs = miss_ratios[~df["file_name"].isin(files)]
test_rds = df[df["file_name"].isin(files)]
test_mrs = miss_ratios[df["file_name"].isin(files)]
print(train_rds)

                         id         0         1         2        3        4  \
0       400.perlbench-41B_0  50816880  17323312  11868112  7701840  3853504   
1     400.perlbench-41B_436  17361520   6023600   4203744  2591488  1828656   
2       400.perlbench-50B_0  58283520  17249296  14592592  9466576  7204896   
3          401.bzip2-226B_0   3098304   3037808   2698720  2692176  1999568   
4         401.bzip2-226B_54   1623056   1618144   1501200  1353728  1014736   
...                     ...       ...       ...       ...      ...      ...   
6184     657.xz_s-3167B_164   5260112   6183536   3447952  2850672  2245888   
6185     657.xz_s-3167B_274   8309920  11139248   5694208  4479968  3585760   
6186     657.xz_s-3167B_466   4869744   5968224   3301024  2821488  2218352   
6187     657.xz_s-3167B_574   4967104   7056944   3530208  2928336  2367040   
6190         657.xz_s-56B_0  26356736  36838224  10074240  2059088  2506384   

            5        6        7        8  ...  3690

In [7]:
train_rds = train_rds.drop(columns=["id", "file_name"])
test_rds = test_rds.drop(columns=["id", "file_name"])

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import MeanSquaredError
from keras.losses import MeanSquaredError
# Create the model
model = Sequential()

# Add the input layer and the first hidden layer
model.add(Dense(896, input_dim=896, activation='sigmoid'))

# Add the second hidden layer
model.add(Dense(896, activation='sigmoid'))

# Add the output layer
model.add(Dense(4, activation='sigmoid'))

# Compile the model
model.compile(loss=MeanSquaredError() , optimizer='adam')

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 896)               803712    
                                                                 
 dense_1 (Dense)             (None, 896)               803712    
                                                                 
 dense_2 (Dense)             (None, 4)                 3588      
                                                                 
Total params: 1,611,012
Trainable params: 1,611,012
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(train_rds.to_numpy(), train_mrs.to_numpy(), epochs=10)

Epoch 1/10
155/155 [==============================] - 2s 8ms/step - loss: 0.0535
Epoch 2/10
155/155 [==============================] - 1s 8ms/step - loss: 0.0379
Epoch 3/10
155/155 [==============================] - 1s 8ms/step - loss: 0.0334
Epoch 4/10
155/155 [==============================] - 1s 8ms/step - loss: 0.0298
Epoch 5/10
155/155 [==============================] - 1s 8ms/step - loss: 0.0271
Epoch 6/10
155/155 [==============================] - 1s 8ms/step - loss: 0.0271
Epoch 7/10
155/155 [==============================] - 1s 8ms/step - loss: 0.0251
Epoch 8/10
155/155 [==============================] - 1s 8ms/step - loss: 0.0242
Epoch 9/10
155/155 [==============================] - 1s 8ms/step - loss: 0.0206
Epoch 10/10
155/155 [==============================] - 1s 8ms/step - loss: 0.0143


In [10]:
#from sklearn.metrics import mean_squared_error, mean_absolute_error
# Compare columns
#for i,j in enumerate(['1024','2048','4096','8192']):
 #   model_out = f.numpy()[:,i]
  #  y_out = y_test.to_numpy()[:,i]
   # print(f'Mean squared error for cache size {j}\t: {mean_squared_error(model_out, y_out)}')

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
f = model(test_rds.to_numpy())
# Compare columns
for i,j in enumerate(['1024','2048','4096','8192']):
    model_out = f.numpy()[:,i]
    y_out = test_mrs.to_numpy()[:,i]
    print(f'Mean squared error for cache size {j}\t: {mean_squared_error(model_out, y_out)}')

Mean squared error for cache size 1024	: 0.011784924929154065
Mean squared error for cache size 2048	: 0.029948703831710406
Mean squared error for cache size 4096	: 0.03848118389036915
Mean squared error for cache size 8192	: 0.03892120673627349
